# MNAD Evaluation

## Libraries import

In [ ]:
import argparse
import os
import torch
import torch.utils.data as data
import torchvision.transforms as T

from data.CustomDataset import CustomImageDataset

## Constants

In [ ]:
DATASET_DIR_SUFFIX = 'images'

## Parameters

In [ ]:
args_dict = {
  "gpus": "1",                            # gpus (set 1 or None)
  "batch_size": 1,                        # batch size for testing
  "h": 256,                               # height of input images
  "w": 256,                               # width of input images
  "c": 3,                                 # channel of input images
  "method": "recon",                      # The target task for anoamly detection  (pred or recon)
  "t_length": 1,                          # length of the frame sequences
  "fdim": 512,                            # channel dimension of the features
  "mdim": 512,                            # channel dimension of the memory items
  "msize": 10,                            # number of the memory items
  "alpha": 0.7,                           # weight for the anomality score
  "th": 0.015,                            # threshold for test updating
  "num_workers": 1,                       # number of workers for the test loader
  "dataset_type": "clean_road",           # type of dataset: clean_road
  "dataset_path": "./dataset",            # directory of data
  "label_path": "./dataset",              # directory of labels
  "label_file": "metadata.csv",           # name of the label file
  "model_path": "model/trained",          # directory of model
  "model_file": "model.pth",              # name of the model file
  "m_items_path": "model/trained",        # directory of memory items
  "m_items_file": "keys.pt"               # name of the memory items file
}

args = argparse.Namespace(**args_dict)

## GPU Configurations

In [ ]:
print(torch.cuda.is_available())
if args.gpus is not None and torch.cuda.is_available():
  print(torch.cuda.is_available())

  print(torch.cuda.device_count())

  print(torch.cuda.current_device())

  print(torch.cuda.device(0))

  print(torch.cuda.get_device_name(0))

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if args.gpus is None:
    gpus = "0"
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus
else:
    gpus = ""
    for i in range(len(args.gpus)):
        gpus = gpus + args.gpus[i] + ","
    os.environ["CUDA_VISIBLE_DEVICES"]= gpus[:-1]

#torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

## Data loading

In [ ]:
test_folder = os.path.join(args.dataset_path, args.dataset_type, 'testing', DATASET_DIR_SUFFIX)
test_label_file = os.path.join(args.label_path, args.dataset_type, args.label_file)

#transform = T.Resize((args.h,args.w))
transform = T.Compose([T.ToTensor(),])

# Loading dataset
test_dataset = CustomImageDataset(test_label_file, test_folder, transform = transform)
test_size = len(test_dataset)

test_batch = data.DataLoader(test_dataset, batch_size = args.batch_size,
                              shuffle=True, num_workers=args.num_workers, drop_last=False)
batch_size = len(test_batch)

## Model loading

In [ ]:
# Loading the trained model
model_file = os.path.join(args.model_path, args.dataset_type, args.model_file)
m_items_file = os.path.join(args.m_items_path, args.dataset_type, args.m_items_file)
#model = torch.load(args.model_dir, map_location=torch.device('cpu'))
model = torch.load(model_file)
model.cuda()
#m_items = torch.load(args.m_items_dir, map_location=torch.device('cpu'))
m_items = torch.load(m_items_file)

## Evaluation